In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# ADD OTHER FOLDERS TO THIS LIST TO ADD THEM TO THE sys.path
modules_to_add = [""]

this_file = os.path.abspath('')

for module in modules_to_add:
    p = Path(this_file).parent / module 
    if p.exists():
        sys.path.append(str(p))
        print(f"ADDED: {p}")
    else:
        print(f"ERROR: {p} doesn't exist")

ADDED: /home/gerardozinno/Desktop/Tesi/Code/mlem


In [3]:
print(sys.path)

['/home/gerardozinno/Desktop/Tesi/Code/mlem/notebooks', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python39.zip', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python3.9', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python3.9/lib-dynload', '', '/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages', '/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/IPython/extensions', '/home/gerardozinno/.ipython', '/home/gerardozinno/Desktop/Tesi/Code/mlem']


In [4]:
from sklearn.datasets import load_iris

In [5]:
data, target = load_iris(return_X_y=True, as_frame=True)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

## Fitting a model

In [7]:
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data_scaled, target.to_numpy(), test_size=0.2, random_state=123)

In [9]:
class IrisScaledDataset(Dataset):
    def __init__(self, features, target):
        assert(len(features) == len(target))
        self.feat = features.astype(np.float32)
        self.targ = target
    def __len__(self):
        return len(self.feat)
    
    def __getitem__(self, idx):
        f = self.feat[idx]
        t = self.targ[idx]
        return f,t

In [10]:
train_data = IrisScaledDataset(X_train, y_train)
test_data = IrisScaledDataset(X_test, y_test)

train_dataloader = DataLoader(train_data, batch_size=16)
test_dataloader = DataLoader(test_data, batch_size=16)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x
    
model     = Model(X_train.shape[1])
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()
model

Model(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=3, bias=True)
)

In [13]:
from blackboxes.pytorch.utilities import train

In [14]:
train(model, optimizer, loss_fn,train_dataloader, test_dataloader, epochs=20, device=device)

Epoch: 0, Training Loss: 1.10, Test Loss: 1.066543, accuracy = 0.67
Epoch: 1, Training Loss: 1.07, Test Loss: 1.024893, accuracy = 0.77
Epoch: 2, Training Loss: 1.03, Test Loss: 0.974698, accuracy = 0.83
Epoch: 3, Training Loss: 0.99, Test Loss: 0.911596, accuracy = 0.83
Epoch: 4, Training Loss: 0.94, Test Loss: 0.844105, accuracy = 0.90
Epoch: 5, Training Loss: 0.89, Test Loss: 0.785849, accuracy = 0.90
Epoch: 6, Training Loss: 0.85, Test Loss: 0.745307, accuracy = 0.90
Epoch: 7, Training Loss: 0.82, Test Loss: 0.720292, accuracy = 0.90
Epoch: 8, Training Loss: 0.80, Test Loss: 0.704561, accuracy = 0.90
Epoch: 9, Training Loss: 0.78, Test Loss: 0.693885, accuracy = 0.93
Epoch: 10, Training Loss: 0.76, Test Loss: 0.685195, accuracy = 0.93
Epoch: 11, Training Loss: 0.74, Test Loss: 0.676212, accuracy = 0.97
Epoch: 12, Training Loss: 0.73, Test Loss: 0.666375, accuracy = 1.00
Epoch: 13, Training Loss: 0.71, Test Loss: 0.656841, accuracy = 1.00
Epoch: 14, Training Loss: 0.70, Test Loss: 0

In [16]:
from mlem.black_box import PyTorchBlackBox

bb = PyTorchBlackBox(model)

In [21]:
bb.predict(X_test[:20]) == y_test[:20]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [22]:
bb.predict_proba(X_test[:4])

array([[2.1957760e-03, 5.0833678e-01, 4.8946741e-01],
       [1.2547681e-04, 2.7490411e-02, 9.7238410e-01],
       [6.3781699e-05, 1.4141833e-02, 9.8579431e-01],
       [6.9175966e-02, 8.7685829e-01, 5.3965751e-02]], dtype=float32)

In [15]:
ROOT = Path("../pretrained/")
MODEL_NAME      = "iris_pytorch.tar"
SAVE_PATH = ROOT / MODEL_NAME